In [1]:
# Mount path for Google Colab
from google.colab import drive
drive.mount('/content/drive/')

root_path = '/content/drive/My Drive/comp90042-project1/project-data/'

Mounted at /content/drive/


In [2]:
# Installing libraries
!pip install torch torchvision transformers

# Load pretrained bert base model
# from transformers import BertModel

# bert_model = BertModel.from_pretrained('bert-base-uncased')

# print("Done loading BERT model.")

     |████████████████████████████████| 2.1MB 14.6MB/s 
     |████████████████████████████████| 3.3MB 44.6MB/s 
     |████████████████████████████████| 901kB 55.4MB/s 


In [3]:
# Inspect train data using pandas
import pandas as pd

train_df = pd.read_json(root_path + 'train.data.jsonl', lines=True)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345
0,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated

In [3]:
# Build a preprocessing function for PyTorch DataLoader
import json
import pandas as pd
import re

import torch
from torch.utils.data import Dataset

from sklearn import preprocessing

from transformers import AutoTokenizer

class PreprocessDataset(Dataset):

    def __init__(self, datafile, labelfile, maxlen):
        # Define constants
        self.REPLY_ID = 'in_reply_to_status_id_str'
        self.TEXT = 'text'

        # Initialise label encoder
        self.le = preprocessing.LabelEncoder()

        # Store train dataset in pandas dataframe
        self.train_df = pd.read_json(datafile, lines=True)

        # Store labels as a list encoded with 0 and 1
        # If labels == None, then preprocessing is done for test dataset
        if labelfile != None:
            with open(labelfile, 'r') as f:
                load_labels = json.load(f)
                self.labels = self.le.fit_transform(list(load_labels.values())).tolist()
        else:
            self.labels = None
        
        # Initialise BERT tokenizer using AutoTokenizer
        self.bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

        # The max length of the tokens
        self.maxlen = maxlen
    
    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, index):
        # Start preprocessing train_df
        source = ''
        replies = ''
        row_tweet = list(self.train_df.iloc[index])

        for tweet in row_tweet:
            if tweet == None:
                break
            if tweet[self.REPLY_ID] == None:
                source = tweet[self.TEXT]
            else:
                tweet_text = tweet[self.TEXT]
                # Remove links
                # Link: https://www.codegrepper.com/code-examples/python/remove+urls+from+text+python
                tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', tweet_text, flags=re.MULTILINE)
                # Remove twitter handle
                # Link: https://stackoverflow.com/questions/50830214/remove-usernames-from-twitter-data-using-python
                # tweet_text = re.sub('@[^\s]+', '', tweet_text, flags=re.MULTILINE)
                # Remove just the hashtag
                # tweet_text = tweet_text.replace('#', '')
                replies += tweet_text + ' '
        
        bert_tokens = self.bert_tokenizer(source,
                                          replies,
                                          padding='max_length',
                                          truncation=True,
                                          max_length=self.maxlen)
        
        # Example output of bert_tokens
        # {'input_ids': [101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102],
        # 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        # 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        
        if self.labels != None:
            label = self.labels[index]
        else:
            label = None

        tokens_ids_t = torch.tensor(bert_tokens['input_ids'])
        attn_mask_t = torch.tensor(bert_tokens['attention_mask'])
        seg_ids_t = torch.tensor(bert_tokens['token_type_ids'])

        if label == None:
            return tokens_ids_t, attn_mask_t, seg_ids_t


        return tokens_ids_t, attn_mask_t, seg_ids_t, label


In [4]:
from torch.utils.data import DataLoader

MAX_LEN = 512
BATCH_SIZE = 10

# creating instances of training and development set
train_set = PreprocessDataset(datafile=root_path + 'train.data.jsonl',
                              labelfile=root_path + 'train.label.json',
                              maxlen=MAX_LEN)
dev_set = PreprocessDataset(datafile=root_path + 'dev.data.jsonl',
                            labelfile=root_path + 'dev.label.json',
                            maxlen=MAX_LEN)

# creating instances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=2)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, num_workers=2)

print('Done preprocessing')


Done preprocessing


In [4]:
import torch
import torch.nn as nn
from transformers import BertModel

class RumourClassifier(nn.Module):

    def __init__(self):
        super(RumourClassifier, self).__init__()

        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')

        # classification layer
        self.cls_layer = nn.Linear(768, 1)
    
    def forward(self, seq, attn_masks, seg_ids):
        outputs = self.bert_layer(seq,
                                  attention_mask=attn_masks,
                                  token_type_ids=seg_ids)
        cont_reps = outputs.last_hidden_state

        cls_rep = cont_reps[:, 0]

        logits = self.cls_layer(cls_rep)

        return logits

In [5]:
gpu = 0

print("Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...")
net = RumourClassifier()
net.cuda(gpu) # Enable gpu support for the model
print("Done creating the rumour classifier.")

Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...



Done creating the rumour classifier.


In [6]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [7]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, seg_ids, labels in dataloader:
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks, seg_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [8]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
            # clear gradients
            opti.zero_grad()
            # converting these to cuda tensors
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks, seg_ids)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
            
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        print("Saving model..")
        torch.save(net.state_dict(), root_path + 'ica_rumor_cls_link_removed_{}.dat'.format(ep))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}".format(best_acc, dev_acc))
            best_acc = dev_acc

In [10]:
num_epoch = 4

train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.6207184195518494; Accuracy: 0.699999988079071; Time taken (s): 1.2445266246795654
Iteration 100 of epoch 0 complete. Loss: 0.8203243613243103; Accuracy: 0.5; Time taken (s): 99.58915877342224
Iteration 200 of epoch 0 complete. Loss: 0.18863897025585175; Accuracy: 1.0; Time taken (s): 101.84353280067444
Iteration 300 of epoch 0 complete. Loss: 0.593815803527832; Accuracy: 0.800000011920929; Time taken (s): 102.415700674057
Iteration 400 of epoch 0 complete. Loss: 0.6243154406547546; Accuracy: 0.800000011920929; Time taken (s): 102.57909393310547
Epoch 0 complete! Development Accuracy: 0.8620690703392029; Development Loss: 0.31076141556018383
Saving model..
Best development accuracy improved from 0 to 0.8620690703392029
Iteration 0 of epoch 1 complete. Loss: 0.2919139564037323; Accuracy: 0.800000011920929; Time taken (s): 89.32256317138672
Iteration 100 of epoch 1 complete. Loss: 0.7484080195426941; Accuracy: 0.6000000238418579; Time taken (s): 10

In [9]:
torch.load(root_path + 'ica_rumor_cls_link_removed_1.dat')

OrderedDict([('bert_layer.embeddings.position_ids',
              tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
                  

In [10]:
import json
from torch.utils.data import DataLoader

def generate_prediction(dataset, maxlen):

    REPLY_ID = 'in_reply_to_status_id_str'
    ID = 'id_str'

    predictions = dict()
    id_list = []
    preds = []

    test_df = pd.read_json(dataset, lines=True)

    test_set = PreprocessDataset(datafile=dataset,
                                 labelfile=None,
                                 maxlen=maxlen)
    
    test_loader = DataLoader(test_set)

    with torch.no_grad():
        for seq, attn_masks, seg_ids in test_loader:
            seq, attn_masks, seg_ids = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu)
            logits = net(seq, attn_masks, seg_ids)
            probs = torch.sigmoid(logits)
            soft_prob = (probs > 0.5).long()
            decoded_pred = 'non-rumour' if soft_prob.squeeze().item() == 0 else 'rumour'
            preds.append(decoded_pred)
    
    for index, row in test_df.iterrows():
        for tweet in row:
            if tweet[REPLY_ID] == None:
                id_list.append(tweet[ID])
                break
    
    for id, pred in zip(id_list, preds):
        predictions[id] = pred
    
    with open(root_path + 'test-output.json', 'w') as outfile:
        json.dump(predictions, outfile)
    
    print('predictions generated!!')

In [11]:
MAX_LEN = 512

generate_prediction(root_path + 'test.data.jsonl', MAX_LEN)


predictions generated!!


-------------------------------------
# This is for testing

In [ ]:
counter = 0
for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
    if counter > 5:
        break
    counter += 1
    print(it, (seq, attn_masks, seg_ids, labels))

In [ ]:
MAX_LEN = 512


# creating instances of training and development set
test_set = PreprocessDataset(datafile=root_path + 'test.data.jsonl',
                             labelfile=None,
                              maxlen=MAX_LEN)

# creating instances of training and development dataloaders
test_loader = DataLoader(train_set)

with torch.no_grad():
    for seq, attn_masks, seg_ids, labels in test_loader:
        seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)
        logits = net(seq, attn_masks, seg_ids)
        probs = torch.sigmoid(logits)
        soft_prob = (probs > 0.5).long()
        decoded_pred = soft_prob.squeeze().item()
        print(decoded_pred)


# prediction = net()

In [ ]:
# Preprocessing attempt
import json
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define constant
REPLY_ID = 'in_reply_to_status_id_str'
# ID = 'id_str'
TEXT = 'text'
# TIMESTAMP = 'created_at'
MAX_LEN = 512

count = 0

bert_X = []

for index, row in train_df.iterrows():
  source_bool = False
  source_tweet = ''
  reply_tweets = ''

  for tweet in row:
    if tweet == None:
      break
    if tweet[REPLY_ID] == None:
      source_tweet = tweet[TEXT]
    else:
      reply_tweets += tweet[TEXT] + ' '
  
  bert_tokens = bert_tokenizer(source_tweet, reply_tweets, padding=True, truncation=True, max_length=512)
  bert_X.append(bert_tokens)
  # print(bert_tokens)
  # count += 1
  # if count > 2:
  #   break

In [36]:
# Simplify replies
# Preprocessing attempt
import json
from transformers import AutoTokenizer
import re as re
from nltk.tokenize import TweetTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
twt_tokenizer = TweetTokenizer()

# Define constant
REPLY_ID = 'in_reply_to_status_id_str'
# ID = 'id_str'
TEXT = 'text'
# TIMESTAMP = 'created_at'
MAX_LEN = 512

count = 0

bert_X = []

row_tweet = list(train_df.iloc[0])
for tweet in row_tweet:
    if tweet == None:
        break
    # tweet_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet[TEXT]).split())
    tweet_text = tweet[TEXT]
    tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', tweet_text, flags=re.MULTILINE)
    tweet_text = re.sub('@[^\s]+', '', tweet_text, flags=re.MULTILINE)
    tweet_text = tweet_text.replace('#', '')
    print(tweet[TEXT])
    print(tweet_text)
    bert_token = bert_tokenizer(tweet_text)
    token_uncleaned = bert_tokenizer(tweet[TEXT])
    print(bert_tokenizer.decode(token_uncleaned['input_ids']))
    print(bert_tokenizer.decode(bert_token['input_ids']))

# for index, row in train_df.iterrows():
#   source_bool = False
#   source_tweet = ''
#   reply_tweets = ''

#   for tweet in row:
#     if tweet == None:
#       break
#     if tweet[REPLY_ID] == None:
#       source_tweet = tweet[TEXT]
#     else:
#       reply_tweets += tweet[TEXT] + ' '
  
#   bert_tokens = bert_tokenizer(source_tweet, reply_tweets, padding=True, truncation=True, max_length=512)
#   bert_X.append(bert_tokens)
  # print(bert_tokens)
  # count += 1
  # if count > 2:
  #   break


How to respond to the murderous attack on Charlie Hebdo? Every newspaper in the free world should print this. http://t.co/sC2ot63F6j
How to respond to the murderous attack on Charlie Hebdo? Every newspaper in the free world should print this. 
[CLS] how to respond to the murderous attack on charlie hebdo? every newspaper in the free world should print this. http : / / t. co / sc2ot63f6j [SEP]
[CLS] how to respond to the murderous attack on charlie hebdo? every newspaper in the free world should print this. [SEP]
@Heresy_Corner @KrustyAllslopp 
Jews label anyone they don't like as Anti-Semite and campaign until that person/company is finished.
  
Jews label anyone they don't like as Anti-Semite and campaign until that person/company is finished.
[CLS] @ heresy _ corner @ krustyallslopp jews label anyone they don't like as anti - semite and campaign until that person / company is finished. [SEP]
[CLS] jews label anyone they don't like as anti - semite and campaign until that person / com

In [ ]:
# Testing bert
bert_model = BertModel.from_pretrained('bert-base-uncased')

outputs = bert_model(train_set.__getitem__(2)[0],
                     attention_mask=train_set.__getitem__(2)[1],
                     token_type_ids=train_set.__getitem__(2)[2])
hidden_reps = outputs.last_hidden_state
print(hidden_reps.shape)

torch.Size([1, 458, 768])


In [ ]:
import json
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

senc = ['this is a sen', 'this is another', 'and another one']
# print(senc)
senc_tokens = bert_tokenizer(senc)
print(senc_tokens)

{'input_ids': [[101, 2023, 2003, 1037, 12411, 102], [101, 2023, 2003, 2178, 102], [101, 1998, 2178, 2028, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}


In [ ]:
import json
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

REPLY_ID = 'in_reply_to_status_id_str'
ID = 'id_str'

train_id = []

for index, row in train_df.iterrows():
  for tweet in row:
    if tweet[REPLY_ID] == None:
      train_id.append(tweet[ID])
      break
  
with open(root_path + 'train.label.json', 'r') as f:
  train_labels = json.load(f)
  label_df = pd.DataFrame.from_dict(train_labels, orient='index', columns=['label'])
  label_id = le.fit_transform(list(train_labels.values())).tolist()
  label_df = pd.DataFrame(label_id)

print(label_id)
print(le.classes_)
print(label_df)
# print(label_df)

def Diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

# print(Diff(train_id, label_id))

[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 

In [ ]:
index_test = []

for index, row in train_df.iterrows():
  source_index = 0
  for tweet in row:
    source_index += 1
    if tweet[REPLY_ID] == None:
      index_test.append(source_index)
      break

print(index_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 